In [31]:
import os
import numpy as np

from flow.scenarios.merge import MergeScenario, ADDITIONAL_NET_PARAMS
from flow.core.params import VehicleParams, NetParams, InitialConfig, TrafficLightParams, InFlows, SumoParams, EnvParams, SumoCarFollowingParams
from flow.envs.merge import WaveAttenuationMergePOEnv, ADDITIONAL_ENV_PARAMS as WA_ADDITIONAL_ENV_PARAMS
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS as LA_ADDITIONAL_ENV_PARAMS
from flow.core.experiment import Experiment

In [32]:
%load_ext autoreload
%autoreload 2
from merge import NashController 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Add Vehicles to the Scenario
These vehicle will be controlled by whatever acceleration controller set to.

In [33]:
name = "merge_example"
vehicles = VehicleParams()
vehicles.add("human",
             acceleration_controller=(NashController, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="aggressive"))

## Set the Net Params
These control the configuration of the highway merge network.

In [34]:
# Optionally adjust the net params
additional_net_params = ADDITIONAL_NET_PARAMS.copy()
additional_net_params['merge_length'] = 50 # length of the merge edge
additional_net_params['pre_merge_length'] = 50 # length of the highway leading to the merge
additional_net_params['post_merge_length'] = 100  # length of the highway past the merge
additional_net_params['merge_lanes'] = 1 # number of lanes in the merge
additional_net_params['highway_lanes'] = 1 # number of lanes in the highway
additional_net_params['speed_limit'] = 30 # max speed limit of the network

## Add Inflows to the Scenario

In [35]:
inflow = InFlows()

# Inflow for highway lanes
inflow.add(veh_type="human",
           edge="inflow_highway",
           probability=1.0, #0.5
           begin=1,
           number=2, # 100
           depart_speed=additional_net_params['speed_limit'],
           color="red")

# Inflow for merge lane
inflow.add(veh_type="human",
           edge="inflow_merge",
           probability=1.0, #0.1
           depart_speed="random",
           begin=1, 
           number=1, # 30
           color="blue")

net_params = NetParams(inflows=inflow,
                       additional_params=additional_net_params)

## Set the Initial Configuration Params
These parameters that affect the positioning of vehicle in the network at the start of a rollout. By default, vehicles are uniformly distributed in the network.

In [36]:
initial_config = InitialConfig(shuffle=False,
                               spacing="uniform",
                               min_gap=0,
                               perturbation=1,
                               x0=0,
                               bunching=0,
                               lanes_distribution=float("inf"),
                               edges_distribution="all",
                               additional_params=None)

## Set the Traffic Light Params
This class is used to place traffic lights in the network and describe the state of these traffic lights. In addition, this class supports modifying the states of certain lights via TraCI.

In [37]:
traffic_lights = TrafficLightParams()

## Set the Simulation Params
These include the simulation time step, whether or not to render the simulation, and where to write the output results.

In [38]:
sumo_params = SumoParams(sim_step=0.1, render=True, emission_path='data')

## Set the Environment Params
We can use the WaveAttenuationEnv or the AccelEnv.

In [39]:
# Optionally adjust the env params
USE_WA_ENV = False

additional_env_params = None
if USE_WA_ENV:
    additional_env_params = WA_ADDITIONAL_ENV_PARAMS.copy()
    additional_env_params['max_accel'] = 3
    additional_env_params['max_decel'] = 3
    additional_env_params['target_velocity'] = 25
    additional_env_params['num_rl'] = 5
else:
    additional_env_params = LA_ADDITIONAL_ENV_PARAMS.copy()
    additional_env_params['max_accel'] = 3
    additional_env_params['max_decel'] = 3
    additional_env_params['target_velocity'] = 25
    additional_env_params['sort_vehicles'] = False

env_params = EnvParams(additional_params=additional_env_params,
                       horizon=float('inf'),
                       warmup_steps=0,
                       sims_per_step=1,
                       evaluate=False,
                       clip_actions=True)

## Run the Experiment

In [40]:
# create the scenario object
scenario = MergeScenario(name="merge_example",
                         vehicles=vehicles,
                         net_params=net_params,
                         initial_config=initial_config,
                         traffic_lights=traffic_lights)

# create the environment object
env = None
if USE_WA_ENV:
    env = WaveAttenuationMergePOEnv(env_params, sumo_params, scenario)
else:
    env = AccelEnv(env_params, sumo_params, scenario)

# create the experiment object
exp = Experiment(env)

# run the experiment for a set number of rollouts / time steps
_ = exp.run(1, 3000, convert_to_csv=True)

**********************************************************
**********************************************************
**********************************************************
significantly decrease after large number of rollouts. In 
order to avoid this, set SumoParams(restart_instance=True).
**********************************************************
**********************************************************
**********************************************************
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      152
Number of nonzeros in inequality constraint Jacobian.:       40
Number of nonzeros in Lagrangian Hessian.............:     1830

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0

/home/neerajb/anaconda3/envs/flow/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/neerajb/anaconda3/envs/flow/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      152
Number of nonzeros in inequality constraint Jacobian.:       40
Number of nonzeros in Lagrangian Hessian.............:     1830

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       38
Total number of inequality constraints...............:       40
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:       40
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:159:
.../casadi/core/optistack_internal.cpp:999: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Infeasible_Problem_Detected'

In [ ]:
emission_location = os.path.join(exp.env.sim_params.emission_path, exp.env.scenario.name)
print(emission_location + '-emission.csv')